In [27]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
from IPython.display import display, HTML
import math

In [28]:
def selecionar_aprovados(dados):

    df = dados.copy(deep=True)

    # Seleciona os 81 primeiros candidatos aprovados
    df.loc[df.loc[df["APROVADO DISCURSIVA"] == True].head(81).index, 'SITUAÇÃO PARCIAL'] = "Aprovado nas vagas de ampla concorrência"

    # Seleciona os candidatos aprovados PCD que ainda não foram incluídos
     
    df.loc[df.loc[(df['SITUAÇÃO PARCIAL'].isna()) & 
                  (df["SITUACAO OBJETIVA"] == "Aprovado PcD") &
                  (df["APROVADO DISCURSIVA"] == True)].head(9).index, 'SITUAÇÃO PARCIAL'] = "Aprovado nas vagas de cota PCD"

    # Caso não haja 9 candidatos aprovados PCD, completa com aprovados na Ampla
    if len(df[df['SITUAÇÃO PARCIAL'] == "Aprovado nas vagas de cota PCD"]) < 9:
        df.loc[df.loc[(df['SITUAÇÃO PARCIAL'].isna()) &
                      (df["APROVADO DISCURSIVA"] == True)].head(9 - len(df[df['SITUAÇÃO PARCIAL'] == "Aprovado nas vagas de cota PCD"])).index, 'SITUAÇÃO PARCIAL'] = "Aprovado nas vagas de ampla concorrência"

    df.loc[(df['SITUAÇÃO PARCIAL'].isna()) & 
           (df["APROVADO DISCURSIVA"] == True), 'SITUAÇÃO PARCIAL'] = "Cadastro de Reservas"

    df['SITUAÇÃO PARCIAL'] = df['SITUAÇÃO PARCIAL'].fillna('Reprovado na discursiva')
      
    # Retorna os 81 primeiros aprovados + os 9 primeiros aprovados PCD
    return df

In [29]:
def gerar_html_linha(row):
    return f'''<tr style="background-color: rgba({'193, 225, 193' if (row['SITUAÇÃO PARCIAL'] == "Aprovado nas vagas de ampla concorrência") | (row['SITUAÇÃO PARCIAL'] == "Aprovado nas vagas de cota PCD") else
     '255, 250, 160' if row['SITUAÇÃO PARCIAL'] == "Cadastro de Reservas" else 
     '250, 160, 160'},{'1' if row['POS'] % 2 == 0 else '0.8'});">
    <td>{row['POS']}</td>
    <td>{row['INSC_NUMB']}</td>
    <td>{row['NOME']}</td>
    <td>{row['TOTAL']}</td>
    <td>{'SIM' if row['IDOSO'] else 'NÃO'}</td>
    <td>{row['TOTAL DISCURSIVA'] if ~np.isnan(row['TOTAL DISCURSIVA']) else '-'}</td>
    <td>{row['P2']}</td>
    <td>{row['CD']}</td>
    <td>{row['DEV']}</td>
    <td>{row['BD']}</td>
    <td>{row['NASC'].strftime('%d/%m/%Y')}</td>
    <td>{'<b>⇧ </b>' + str(row['POS OBJETIVA'] - row['POS']) if row['POS OBJETIVA'] > row['POS'] else 
         '-' if row['POS OBJETIVA'] == row['POS'] else
         '<b>⇩ </b>' + str(row['POS'] - row['POS OBJETIVA'])}</td>
    <td>{row['SITUAÇÃO PARCIAL']}</td>
    </tr>'''

In [30]:
resultado_objetiva = pd.read_csv('./data/resultado-objetiva.csv', sep=";", index_col=['INSC_NUMB', 'NOME'])
resultado_discursiva = pd.read_csv('./data/resultado-discursiva.csv', sep=";", index_col=['INSC_NUMB', 'NOME'])

In [31]:
resultado_preliminar = resultado_objetiva.merge(resultado_discursiva, on=['INSC_NUMB', 'NOME'], how='left')

In [32]:
resultado_preliminar['TOTAL'] = resultado_preliminar['TOTAL OBJETIVA'] + resultado_preliminar['TOTAL DISCURSIVA'].fillna(value=0)
resultado_preliminar['APROVADO DISCURSIVA'] = resultado_preliminar['TOTAL DISCURSIVA'] >= 40
resultado_preliminar['NASC'] = pd.to_datetime(resultado_preliminar['NASC'], format='%d/%m/%Y')
today = datetime.today()
resultado_preliminar['IDADE'] = (today - resultado_preliminar['NASC']).astype('timedelta64[Y]')
resultado_preliminar['IDOSO'] = resultado_preliminar['IDADE'] >= 60

In [33]:
resultado_preliminar_ordenado = resultado_preliminar.sort_values(by=['APROVADO DISCURSIVA', 'TOTAL', 'IDOSO', 'TOTAL DISCURSIVA', 'P2', 'CD', 'DEV', 'BD', 'NASC'], 
                                               ascending=[False, False, False, False, False, False, False, False, True])

In [34]:
resultado_preliminar_ordenado = resultado_preliminar_ordenado.reset_index(drop=False)
resultado_preliminar_ordenado['POS'] = resultado_preliminar_ordenado.index + 1

In [35]:
resultado_preliminar_ordenado = selecionar_aprovados(resultado_preliminar_ordenado)

In [36]:
resultado_preliminar_ordenado['html_linha'] = resultado_preliminar_ordenado.apply(gerar_html_linha, axis=1)

In [37]:
render_html = '''<style>
table {
  border-collapse: collapse;
  font-size: 12px;
}
table, th, td {
  border: 1px solid black;
  text-align: center;
}
th {
  font-weight: bold;
  top: 0;
}
body {
  font-family: Arial, sans-serif;
}
thead { 
  display: table-header-group; 
}
@media print {
  tr {
    page-break-inside: avoid;
  }
}
@page {
  size: A3;
}
</style>
<h4><center>Resultado Preliminar - Tecnologia da Informação<br>Secretaria de Estado de Fazenda de Minas Gerais</center></h4>
Critérios de desempate:<br>
a) tiver idade igual ou superior a sessenta anos, nos termos do art. 27, parágrafo único, do Estatuto do Idoso;<br>
b) obtiver maior nota na Prova Escrita Discursiva;<br>
c) obtiver maior nota na Prova Escrita Objetiva II;<br>
d) obtiver maior número de acertos em Ciência de Dados;<br>
e) obtiver maior número de acertos em Desenvolvimento de Sistemas;<br>
f) obtiver maior número de acertos em Banco de Dados;<br>
g) tiver exercido a função de jurado, nos termos do art. 440 do Código de Processo Penal (não considerado nesta classificação);<br>
h) persistindo o empate, observar-se-á o critério de maior idade.
<table style="margin-top: 10px;">
  <colgroup>
    <col style="width: 7%;">
    <col style="width: 7%;">
    <col style="width: 24%;">
    <col style="width: 5%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 4%;">
    <col style="width: 9%;">
    <col style="width: 8%;">
    <col style="width: 16%;">
  </colgroup>
  <thead>
<tr>
<th rowspan="2">Posição preliminar</th>
<th rowspan="2">Inscrição</th>
<th rowspan="2">Nome</th>
<th rowspan="2">Nota total preliminar</th>
<th colspan="7">Critérios de desempate</th>
<th rowspan="2">Variação de posição</th>
<th rowspan="2">Situação preliminar</th>
</tr>
<tr>
    <th>Idoso</th>
    <th>P3</th>
    <th>P2</th>
    <th>CD</th>
    <th>DS</th>
    <th>BD</th>
    <th>Data de Nascimento</th>
</tr>
</thead>'''

for index, row in resultado_preliminar_ordenado.iterrows():
    render_html += row['html_linha']

render_html += '''</table>'''

In [38]:
with open('./output/resultado_preliminar.html', 'w') as f:
    f.write(render_html)

In [39]:
display(HTML(render_html))